In [1]:
import nltk
from nltk import sent_tokenize 
from nltk import word_tokenize
from nltk import PorterStemmer
from nltk import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import TweetTokenizer
from sklearn.feature_extraction.text import CountVectorizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

#ML models, and other tools
import torch as torch
from torch import nn as nn
import keras as keras
from keras.preprocessing.text import Tokenizer


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import os

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Denylson\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Denylson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Denylson\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# X_train, X_test, Y_train, Y_test = train_test_split(
#     df['tweet_token'],
#     df['class'],
#     test_size = 0.2,
#     random_state = 10,
#     stratify=df['class'].values,
# )

In [3]:
#Creating glove to then use for overlapping with our dataset
import torchtext

#glove is a pre-trained library of vocabulary that has every word given thier corresponding identity number
glove = torchtext.vocab.GloVe(name='6B',dim=50)
glove.itos

['the',
 ',',
 '.',
 'of',
 'to',
 'and',
 'in',
 'a',
 '"',
 "'s",
 'for',
 '-',
 'that',
 'on',
 'is',
 'was',
 'said',
 'with',
 'he',
 'as',
 'it',
 'by',
 'at',
 '(',
 ')',
 'from',
 'his',
 "''",
 '``',
 'an',
 'be',
 'has',
 'are',
 'have',
 'but',
 'were',
 'not',
 'this',
 'who',
 'they',
 'had',
 'i',
 'which',
 'will',
 'their',
 ':',
 'or',
 'its',
 'one',
 'after',
 'new',
 'been',
 'also',
 'we',
 'would',
 'two',
 'more',
 "'",
 'first',
 'about',
 'up',
 'when',
 'year',
 'there',
 'all',
 '--',
 'out',
 'she',
 'other',
 'people',
 "n't",
 'her',
 'percent',
 'than',
 'over',
 'into',
 'last',
 'some',
 'government',
 'time',
 '$',
 'you',
 'years',
 'if',
 'no',
 'world',
 'can',
 'three',
 'do',
 ';',
 'president',
 'only',
 'state',
 'million',
 'could',
 'us',
 'most',
 '_',
 'against',
 'u.s.',
 'so',
 'them',
 'what',
 'him',
 'united',
 'during',
 'before',
 'may',
 'since',
 'many',
 'while',
 'where',
 'states',
 'because',
 'now',
 'city',
 'made',
 'like',
 

In [19]:
dataframe = pd.read_csv('./labeled_data.csv')
dataframe = dataframe[['class', 'tweet']]
dataframe = dataframe.replace([0,1,2], ['Hate Speech', 'Offensive Language', 'Other'])
ps = PorterStemmer()
lamatizer = WordNetLemmatizer()

stops = set(stopwords.words("english"))
dataframe = dataframe.apply(lambda x: x.astype(str).str.lower())
dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')
dataframe.tweet = dataframe.tweet.str.replace('[^\s\w]','')
dataframe.tweet = dataframe.tweet.str.replace('rt', '')
dataframe['tweet_token'] = dataframe['tweet'].apply(lambda x: word_tokenize(x))
# dataframe['tweet_recon'] = dataframe['tweet_token'].apply(lambda x: list(ps.stem(i) for i in x))
# dataframe['tweet_recon'] = dataframe['tweet_recon'].apply(lambda x: ' '.join(list(i for i in x if i not in stops)))
# tweet_tokenizer = Tokenizer(num_words = 4500, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ') 
# tweet_tokenizer.fit_on_texts(texts = dataframe['tweet_token'])

# dataframe['tweet_token'] = tweet_tokenizer.texts_to_sequences(texts = dataframe['tweet_token'])

dataframe.to_csv('./new_data.csv')

C:\Users\Denylson\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == '__main__':
C:\Users\Denylson\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  # Remove the CWD from sys.path while we load stuff.


In [30]:
from collections import Counter
class Merger():
    
    def __init__(self, csv_path, max_length):
        self.dataframe = pd.read_csv(csv_path)
        self.max_length = max_length
        self.label_dict = {
            "hate speech" : 0,
            "offensive language" : 1,
            "other": 2
        }
        self.dataframe['tweet_token'] = self.dataframe['tweet'].apply(lambda x: word_tokenize(x))
        
        all_mentioned_words = []
        for words in dataframe['tweet_token']:
            all_mentioned_words += words
        frequency = Counter(all_mentioned_words)
        self.vocab = torchtext.vocab.Vocab(counter = frequency, min_freq = 25, vectors = glove)
        
        
#         print(self.dataframe.head())

    def __len__(self):
        return len(self.dataframe['tweet'])
    
    def back_to_text(self, tokens):
        text = ''
        for tok in tokens:
            text += self.vocab.itos[tok] + " "
        return text
    
    
    def __getitem__(self, index):
        label = self.label_dict[self.dataframe['class'][index]]
        label = torch.tensor(label)
        int_tokens = []
        tweet_tokens = self.dataframe['tweet_token'][index]
        for token in tweet_tokens:
            int_tokens.append(self.vocab[token])
        if(len(int_tokens) < self.max_length):
            num_to_pad = self.max_length - len(int_tokens)
            int_tokens += [0] * num_to_pad
        else:
            int_tokens = int_tokens[:self.max_length]
        int_tokens = torch.tensor(int_tokens)
        return(int_tokens, label)
        
            
        
    

In [77]:
dataset = Merger( './new_data.csv', 50)
len(dataset)
pytorch.dataset.subset()

24783

In [85]:
import random
indices_list = list(range(0, 24783))
train_amount = int(0.8* len(dataset))
train_indices = list(range(0, train_amount))
test_indices = list(range(train_amount, len(dataset)))

In [87]:
from torch.utils.data import Subset
train_dataset = Subset(dataset, train_indices)
test_dataset = Subset(dataset, test_indices)

In [89]:
from torch.utils.data import Dataset, DataLoader
train_dataloader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_dataloader = DataLoader(test_dataset, batch_size = 32)
train_dataloader

In [90]:
batched_data, batched_labels = next(iter(train_dataloader))
batched_data.shape, batched_labels.shape

(torch.Size([32, 50]), torch.Size([32]))

In [91]:
class NLP_model(nn.Module):
    def __init__(self, num_words, emb_size, num_classes):
        super().__init__()
        self.num_words = num_words
        self.emb_size = emb_size
        self.emb = nn.Embedding(self.num_words, self.emb_size)
        self.emb.from_pretrained(vocab.vectors)
        self.lstm = nn.LSTM(input_size = emb_size, hidden_size = 32, batch_first = True, num_layers = 2)
        self.relu = nn.ReLU()
        self.lin = nn.Linear(64, num_classes)
        
    def forward(self, batch_data):
        token_embs = self.emb(batch_data)
        outputs, (h_n, c_n) = self.lstm(token_embs)
        
        last_hidden_state = h_n
        last_hidden_state = last_hidden_state.permute(1, 0, 2)
        last_hidden_state = last_hidden_state.flatten(start_dim = 1)

        last_hidden_state = self.relu(last_hidden_state)
        logits = self.lin(last_hidden_state)
        
        return logits
        

In [103]:
model = NLP_model(num_words = len(vocab), emb_size = 50, num_classes = 3)
model

NLP_model(
  (emb): Embedding(1183, 50)
  (lstm): LSTM(50, 32, num_layers=2, batch_first=True)
  (relu): ReLU()
  (lin): Linear(in_features=64, out_features=3, bias=True)
)

In [104]:
batched_data.shape

torch.Size([32, 50])

In [105]:
preds = model(batched_data)

In [106]:
from torch import optim
opt = optim.Adam(model.parameters(), lr=0.01)
opt

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)

In [107]:
loss_func = nn.CrossEntropyLoss()

In [108]:
loss = loss_func(preds, batched_labels)
loss

tensor(1.2313, grad_fn=<NllLossBackward>)

In [109]:
loss.backward()
opt.step()
opt.zero_grad()
#backwards, opt.step, opt.gradient

In [110]:
preds = model(batched_data)
loss_func(preds, batched_labels)

tensor(1.0296, grad_fn=<NllLossBackward>)

In [111]:
loss_func = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.01)

In [119]:
preds[0]
preds.shape

torch.Size([32, 3])

In [132]:
def cal_acc(preds, batched_labels):
    predicted_classes = torch.softmax(preds, dim = 1).argmax(dim = 1)

    num_correct = (predicted_classes == batched_labels).sum()

    acc = num_correct/len(batched_labels)
    return acc

In [134]:
num_epochs = 2
for epoch in range(num_epochs):
    print("-" * 50)
    for i, (batched_data, batched_labels) in enumerate(train_dataloader):
        preds = model(batched_data)
        loss = loss_func(preds, batched_labels)
        accuracy = cal_acc(preds, batched_labels)
        if (i % 50 == 0):
            print("Train: loss : {0}, accuracy: {1}".format(loss, accuracy))
        loss.backward()
        opt.step()
        opt.zero_grad()
    for i, (batched_data, batched_labels) in enumerate(test_dataloader):
        preds = model(batched_data)
        loss = loss_func(preds, batched_labels)
        if (i % 50 == 0):
            print("Test: loss : {0}, accuracy: {1}".format(loss, accuracy))

--------------------------------------------------
Train: loss : 0.7552304863929749, accuracy: 0.6875
Train: loss : 0.7934530973434448, accuracy: 0.71875
Train: loss : 0.8785288333892822, accuracy: 0.6875
Train: loss : 0.49955829977989197, accuracy: 0.875
Train: loss : 0.7573117613792419, accuracy: 0.75
Train: loss : 0.8236463069915771, accuracy: 0.6875
Train: loss : 0.6129195094108582, accuracy: 0.8125
Train: loss : 0.8169825077056885, accuracy: 0.6875
Train: loss : 0.5343446135520935, accuracy: 0.8125
Train: loss : 0.5847054719924927, accuracy: 0.78125
Train: loss : 0.7786131501197815, accuracy: 0.71875
Train: loss : 0.9142997860908508, accuracy: 0.65625
Train: loss : 0.6334542036056519, accuracy: 0.78125
Test: loss : 0.654453456401825, accuracy: 0.9444444179534912
Test: loss : 0.7114994525909424, accuracy: 0.9444444179534912
Test: loss : 0.5666468739509583, accuracy: 0.9444444179534912
Test: loss : 0.5547423958778381, accuracy: 0.9444444179534912
------------------------------------

In [136]:
params = model.state_dict()
torch.save(params, 'my_model_weights.pt')

In [ ]:
params_loaded = torch.load('my_model_weights.pt')
model2.load_state_dict(params_loaded)